In [1]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [2]:
# Function to get current working directory
def current_path():
    print("Current Directory:")
    print(os.getcwd())
    print()
    
# Function to get new directory 
def new_path():
    print("New Directory:")
    print(os.getcwd())
    print()
 
 
# Driver's code
# Printing CWD before
current_path()
 
# Changing the CWD
os.chdir('../CSV/')
 
# Printing CWD after
new_path()


def get_csv_files():
    # Get the current working directory
    cwd = os.getcwd()

    # Get a list of all files and directories in the current working directory
    files_and_directories = os.listdir(cwd)

    # Filter out only the CSV files
    csv_files = [file for file in files_and_directories if file.endswith('.csv')]

    # Return the list of CSV files
    return csv_files


# Get a list of all CSV files in the current working directory
csv_files = get_csv_files()

# Get the current working directory
cwd = os.getcwd()

# Create an empty dictionary to store the DataFrames
dfs = {}

# Loop over each CSV file and read it into a DataFrame
for file in csv_files:
    full_path = os.path.join(cwd, file)
    df = pd.read_csv(full_path)
    dfs[file] = df

# Print the dictionary of DataFrames
print(dfs)

Current Directory:
C:\Users\E178162\Documents\Work Orders\DB\Jupyter Notebooks\DB Import

New Directory:
C:\Users\E178162\Documents\Work Orders\DB\Jupyter Notebooks\CSV

{'departments.csv':     department_id                           department
0               1                  Building Operations
1               2                              RBM/O&M
2               3                       AV/Electronics
3               4                       Campus Support
4               5                           Electrical
5               6                            Help Desk
6               7  Building and Environmental Sciences
7               8                                  FCA
8               9                              Grounds
9              10                         Housekeeping
10             11                           HVAC - A/C
11             12                      HVAC - Controls
12             13                 HVAC - Refrigeration
13             14           Integrated P

In [3]:
# Upload each dataframe into separate tables
dataframes = [dfs['departments.csv'], dfs['School_Information.csv'], dfs['sc_team.csv'], dfs['work_category.csv']]
table_names = ["departments", "school_information", "sc_team", "work_category"]

def clean_colname(dataframe):
    #force column names to be lower case, no spaces, no dashes
    dataframe.columns = [x.lower().replace(" ", "_") for x in dataframe.columns]
    
    #processing data
    replacements = {
       'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'integer',
        'datetime64': 'timestamp',
        'int32': 'integer'
    }

    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    return col_str, dataframe.columns

In [4]:
def upload_to_db(host, dbname, user, password, col_str, table_name, dataframe, cur):
    conn_string = "host={} dbname={} user={} password={}".format(host, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    print('Opened database successfully')
    
    # Drop table with same name
    cur.execute("DROP TABLE IF EXISTS %s CASCADE;" % (table_name))

    # Create table
    cur.execute("CREATE TABLE %s (%s);" % (table_name, col_str))
    print('{0} was created successfully'.format(table_name)) 
    
    # Insert data into table
    for index, row in dataframe.iterrows():
        values = ', '.join(["'{}'".format(str(i).replace("'", "''")) for i in row.tolist()])
        cur.execute("INSERT INTO %s (%s) VALUES (%s);" % (table_name, ", ".join(dataframe.columns), values))
    print('Data was inserted successfully')
    
    cur.execute("GRANT SELECT ON TABLE %s TO public;" % table_name)
    
    conn.commit()
    print('Table {0} imported to db completed'.format(table_name))
    print()
    
    return 'Upload to database completed successfully'


# Create a cursor object
conn_string = "host=localhost dbname=AE_Work_Orders_NM user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

# Upload each dataframe into separate tables
for i, df in enumerate(dataframes):
    col_str, dataframe_columns = clean_colname(df)
    table_name = table_names[i]
    upload_to_db("localhost", "AE_Work_Orders_NM", "postgres", "postgres", col_str, table_name, df, cur)

# Add primary key to Table 1
cur.execute("ALTER TABLE work_category ADD PRIMARY KEY (work_category_id)")
cur.execute("ALTER TABLE work_category ADD CONSTRAINT unique_work_category UNIQUE (work_category)")

# Add primary key to Table 2
cur.execute("ALTER TABLE sc_team ADD PRIMARY KEY (sc_team_id)")

# Add primary key to Table 3
cur.execute("ALTER TABLE departments ADD PRIMARY KEY (department_id)")

# Add primary key to Table 4
cur.execute("ALTER TABLE school_information ADD PRIMARY KEY (site_id)")

# Add foreign keys to Table 1
cur.execute("ALTER TABLE work_category ADD CONSTRAINT fk_department_id FOREIGN KEY (department_id) REFERENCES departments(department_id)")
cur.execute("ALTER TABLE work_category ADD CONSTRAINT fk_sc_team_id FOREIGN KEY (sc_team_id) REFERENCES sc_team(sc_team_id)")

conn.commit()
cur.close()


Opened database successfully
departments was created successfully
Data was inserted successfully
Table departments imported to db completed

Opened database successfully
school_information was created successfully
Data was inserted successfully
Table school_information imported to db completed

Opened database successfully
sc_team was created successfully
Data was inserted successfully
Table sc_team imported to db completed

Opened database successfully
work_category was created successfully
Data was inserted successfully
Table work_category imported to db completed

